In [1]:

import mph

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import peewee

from modules import comsol, utilites

import mph
import re
import pandas as pd
from tqdm import tqdm

In [2]:
client = mph.start()

In [3]:
model = client.load('models/Kinetic model.mph')

In [ ]:
client.clear()

In [ ]:
def sweep_to_sql(
    model: mph.Model,
    name,
    desc=None,
):
    _, light_sweep = model.outer('Data')
    i = 1
    params = {
        key: float(value)
        for key,
        value in _comsol.model_parameters(model).items()
        if 'light' not in key
    }
    params['name'] = name
    params['desc'] = desc

    notes = []
    for light_value in light_sweep:
        note = {}
        note.update(params)
        note['light'] = light_value
        df = _comsol.evaluate_expressions(model, i)
        note['data'] = df.to_json(index=True)
        notes.append(note)

        i += 1
    return


In [4]:
import itertools

In [5]:
class AbstractStudy:

    def __init__(self, model: mph.Model):
        self.comsol_model = model
        self.study_name = self.__class__.__name__

    # @property
    # def model_parametrs(self):
    #     return self.comsol_model.parameters()
    @property
    def time_end(self):
        return model.parameters()['time_end']

    @property
    def constants(self):
        rule = lambda key: key[0] == 'K'
        return {
            key: value
            for key, value in model.parameters().items()
            if rule(key)
        } #yapf: disable

    @property
    def initial(self):
        rule = lambda key: ('0' in key) or (key in ['light'])
        return {
            key: value
            for key, value in model.parameters().items()
            if rule(key)
        } #yapf: disable

    @property
    def study_node(self):
        node = self.comsol_model / 'studies' / f'{self.study_name}'
        assert node.exists(),f'Node {self.study_name} does not exist'
        return node

    @property
    def solution_node(self):
        node = self.comsol_model / 'solutions' / f'{self.study_name}_Solution'
        assert node.exists(),f'Node {self.study_name}_Solution does not exist'
        return node

    @property
    def dataset_node(self):
        node = self.comsol_model / 'datasets' / f'{self.study_name}_data'
        assert node.exists(),f'Node {self.study_name}_data does not exist'
        return node

    def set_parametrs(self, **kwargs):
        for key, value in kwargs.items():
            self.comsol_model.parameter(name=key, value=value)

    @staticmethod
    def _set_node_properties(node: mph.Node, **kwargs):
        for key, value in kwargs.items():
            node.property(key, value)

    def get_species(self) -> dict:
        model = self.comsol_model
        reaction_node = model / 'physics' / 'Reaction Engineering'
        reaction_node_children = [i.name() for i in reaction_node.children()]

        species = re.findall(
            string='\n'.join(reaction_node_children),
            pattern='Species: (.*)',
        )
        return {specie: f'reaction.c_{specie}' for specie in species}

    def _temporal_eval(
        self,
        outer_number,
        functions={},
    ) -> pd.DataFrame:

        model = self.comsol_model
        functions.update({'time': 't'})
        row_data = model.evaluate(
            list(functions.values()),
            dataset=self.dataset_node.name(),
            outer=outer_number,
        )
        return pd.DataFrame(row_data, columns=list(functions))

    def _solve_one(self):
        self.comsol_model.solve(study=self.study_node.name())


class Generator(AbstractStudy):

    def temporal_eval(
        self,
        functions={},
    ) -> pd.DataFrame:

        return self._temporal_eval(
            outer_number=1,
            functions=functions,
        )

    def solve(self):
        self._solve_one()


def make_comdinations(diap: dict):
    keys = list(diap.keys())
    values = [diap[key] for key in keys]
    combinations = list(itertools.product(*values))
    result = [dict(zip(keys, comb)) for comb in combinations]
    return result


def sweep(combinations,generator):
    
    result=[]
    combinations = tqdm(iterable=combinations)
    for combination in combinations:
        generator.set_parametrs(**combination)
        generator.solve()
        df = generator.temporal_eval(generator.get_species())
        df.loc[:,combination.keys()]=list(combination.values())
        result.append(df)
    return pd.concat(result)  
    


class Estimator(AbstractStudy):

    @property
    def estimation_node(self):
        node = self.comsol_model / 'physics' / 'Reaction Engineering' / 'Estimation'
        assert node.exists(),f'Node Estimation does not exist'
        return node

    @property
    def experiments(self):
        return self.estimation_node.children()

    @property
    def tables(self):
        tables = self.comsol_model / 'tables'
        experiment_tables = [
            node for node in tables.children() if 'Experiment' in node.name()
        ]
        return experiment_tables

    def create_one_experiment(
        self,
        data,
        data_columns,
        experiment_i,
        path=r'D:\WORKS\COMSOL_polymers\Batch\generator_out_short.csv',
    ):
        experiment_name = f'exp{experiment_i}'

        # create experiment
        self.estimation_node.java.create(experiment_name, "Experiment", -1)
        experiment = self.experiments[-1]

        # create table
        table_tag = f"tbl_compreactionest1{experiment_name}"
        table = (model / 'tables').java.create(table_tag, "Table")
        table.label(f"Experiment {experiment_i} Table")
        table.setTableData(data)
        table.active(False)

        # set up parametrs
        variables_dict = {'Time': 't'}
        variables_dict.update(self.get_species())
        variables = [variables_dict[key] for key in data_columns]

        Estimator._set_node_properties(
            node=experiment,
            fileName=path,
            dataColumn=data_columns,
            use=[1] * len(data_columns),
            modelVariable=variables,
        )

    # def create_experiments(self,datas):
    #     for i in range(len(datas)):

    #         datas[i]
    #         self.create_one_experiment(

    #     data,
    #     data_columns,
    #     experiment_i=i,
    #     path=r'D:\WORKS\COMSOL_polymers\Batch\generator_out_short.csv',
    # )
    def clear_experiments(self):
        experiments, tables = self.experiments, self.tables
        for table in tables:
            table.remove()
        for experiment in experiments:
            experiment.remove()

    def solve(self):
        self.estimation_node.toggle('on')
        try:
            self._solve_one()
        finally:
            self.estimation_node.toggle('off')

# Generator

In [6]:
gen = Generator(model)

In [7]:
gen.constants

{'Ke': '1000000000 [l/(mol*s)]',
 'KH': '1000000000 [l/(mol*s)]',
 'Kr': '1000000000 [l/(mol*s)]',
 'Kp': '0.001 [1/s]',
 'KqH': '2000 [l/(mol*s)]',
 'Kdisp': '1000000000 [l/(mol*s)]',
 'Kph': '0.00001 [1/s]',
 'Ks': '2 [l/(mol*s)]',
 'Kd': '0.05 [1/s]',
 'Kc': '1 [l/(mol*s)]',
 'KrD': '1000000000 [l/(mol*s)]'}

In [ ]:
species = {
    key: value
    for key,
    value in gen.get_species().items()
    if key in ['Q', 'DH', 'QHH']
}
df = gen.temporal_eval(species)

In [ ]:
df = df/1000
df['time'] = df['time']*1000

In [ ]:
import plotly.express as px

In [ ]:
def simple_temporal_plot(df:pd.DataFrame):
    fig = px.line(df, x='time',y=[col for col in df.columns if col not in ['time','light']])
    fig.update_layout(
        height=500,
        margin={
            'r': 0, 'l': 0, 't': 0, 'b': 0
        },
        legend=dict(x=-0.1, y=1, xanchor="center"),
    )
    return fig

In [ ]:
simple_temporal_plot(df_gen)


# Sweep

In [ ]:
gen.constants

In [36]:
a=make_comdinations({
    'light':(np.linspace(0.1, 5.1, 11)/1000).round(7)
})

In [41]:
df= sweep(a,gen)

100%|██████████| 11/11 [00:14<00:00,  1.34s/it]


In [42]:
df

,Q,DH,Qm,DHp,QH,D,QHD,QHH,prod,QD,time,light
0,1000.000000,1.000000e+03,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0001
1,999.980470,9.999805e+02,0.019530,0.019530,7.449999e-08,7.449999e-08,0.000000e+00,0.000000e+00,1.953087e-12,1.455078e-17,1.953125e-10,0.0001
2,999.965955,9.999660e+02,0.034045,0.034045,2.427463e-07,2.427463e-07,8.553506e-18,8.553503e-18,3.404619e-12,4.978598e-17,3.404698e-10,0.0001
3,999.936927,9.999369e+02,0.063072,0.063072,1.397627e-06,1.397627e-06,5.756434e-16,5.756430e-16,6.308150e-12,4.555122e-16,6.307845e-10,0.0001
4,999.878878,9.998789e+02,0.121112,0.121112,9.914367e-06,9.914367e-06,5.764842e-14,5.764835e-14,1.217081e-11,6.211388e-15,1.211414e-09,0.0001
...,...,...,...,...,...,...,...,...,...,...,...,...
2017,332.761279,-3.099783e-103,0.005256,0.005256,4.707437e-01,6.002663e-05,3.339151e+02,3.328045e+02,3.330182e+02,4.312137e-02,1.940677e-04,0.0051
2018,332.761076,8.850607e-104,0.005206,0.005206,4.707438e-01,5.888321e-05,3.339148e+02,3.328047e+02,3.330182e+02,4.352527e-02,1.958852e-04,0.0051
2019,332.760874,7.241769e-104,0.005157,0.005157,4.707438e-01,5.777216e-05,3.339144e+02,3.328049e+02,3.330182e+02,4.392918e-02,1.977028e-04,0.0051
2020,332.760671,2.193951e-104,0.005109,0.005109,4.707438e-01,5.669227e-05,3.339141e+02,3.328051e+02,3.330182e+02,4.433308e-02,1.995203e-04,0.0051


# Plots

In [ ]:
import plotly.express as px

In [ ]:
rule = {'Kc': (0, 0.30000)}
diap = ['Kc', 'light']
datas, dfs = get_solves(rule)
result = collect_dfs(datas, dfs, diap)


In [ ]:
df_gen

In [ ]:
result =df_gen
fig = px.line(
    result,
    x="time",
    y="DH",
    animation_frame='light',
    # color='Kc',
    range_y=[0, 1],
)
fig.update_layout(
    height=500,
    margin={
        'r': 0, 'l': 0, 't': 0, 'b': 0
    },
    legend=dict(x=-0.1, y=1, xanchor="left"),
)

# Estimator

In [ ]:
# est =Estimator(model)